# Climate dynamics:
## Predicting the weather: spatial patterns approach
In this part of the climate dynamics tutorial, we will use climate model output to estimate the spatial patterns of climate change.  
First, import everything we need.

In [ ]:
!pip install cartopy
!pip install cftime

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import glob
from scipy.constants import convert_temperature

In [ ]:
!git clone https://github.com/peterukk/IE_CDT_ClimateDynamicsTutorial

Predicting long-term climate change requires using climate model scenarios. We will use two equilibrium climate simulations and interpolate between the two: a 4xCO2 scenario and a pre-industrial scenario. Open the data.

In [ ]:
data_path_PI = "IE_CDT_ClimateDynamicsTutorial/data/"
filename_PI = data_path_PI + "tas_Amon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-204912.nc"
ds_PI = xr.open_dataset(filename_PI)
ds_PI

In [ ]:
data_path_4xCO2  = "IE_CDT_ClimateDynamicsTutorial/data/"
filename_4xCO2 = data_path_4xCO2 + "tas_Amon_UKESM1-0-LL_abrupt-4xCO2_r1i1p1f2_gn_195001-199912.nc"
ds_4xCO2 = xr.open_dataset(filename_4xCO2)
ds_4xCO2

Take the average of time and have a look at the differences between the two simulations.

In [ ]:
# Average over time and get just pattern
surface_temp_PI = ds_PI["tas"].mean(dim="time")
surface_temp_4xCO2 = ds_4xCO2["tas"].mean(dim="time")
surface_temp_4xCO2

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6),
                       subplot_kw={'projection': ccrs.EqualEarth(central_longitude=0)})

axs[0].coastlines()
colormap_PI = axs[0].pcolormesh(surface_temp_PI.lon,
                                surface_temp_PI.lat,
                                surface_temp_PI,
                                vmin=220, vmax=320,
                                cmap="RdBu_r", transform=ccrs.PlateCarree())
axs[0].set_title("Pre-industrial temperature")

axs[1].coastlines()
plt.sca(axs[1])
colormap_4xCO2 = axs[1].pcolormesh(surface_temp_4xCO2.lon,
                                   surface_temp_4xCO2.lat,
                                   surface_temp_4xCO2,
                                   vmin=220, vmax=320,
                                   cmap="RdBu_r", transform=ccrs.PlateCarree())
axs[1].set_title("4xCO2 temperature")

plt.colorbar(colormap_PI, ax=axs,
             location="bottom",
             orientation="horizontal",
             shrink=0.6,
             label="Temperature (K)")


## Pattern scaling
A simple approach to interpolate between these two climates at the year 2100 is called "Pattern Scaling" (Santer, 1990, Mitchell, 2003). This has been used for many years and still holds up fairly well. It assumes that the global warming pattern remains constant and is scaled by a scaler variable, such as global mean temperature. Then, we can estimate the global warming changes as
$$P(x,y) =  \frac{\Delta T}{\Delta T_{ref}} P_{ref}(x,y) $$
where $P_{ref}(x,y)$ is a reference global warming pattern, $\Delta T_{ref}$ is the global mean temperature change in the reference scenario and $\Delta T$ is the global mean temperature change in the new scenario. Note that we have to estimate the latter from somewhere, such as the IPCC report, an energy balance model, a climate model emulator. We will assume that in 2100 the global mean temperature change relative to pre-industrial is ~3K [IPCC, 2021].
1. Take temperature difference between two scenarios to get $P_{ref}$
2. Normalise pattern by global mean temperature change - this gives the pattern of warming per 1K of warming $ \frac{P_{ref}(x,y)}{\Delta T_{ref}} $. The normalized pattern should be centered around 1. Don't forget to take an area weighted mean to account for different grid cell sizes, otherwise there will be larger contributions from higher latitudes.
Hint: After you figure out what the weights should be, you can use [xarray's weighted operations to store them and apply them](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.weighted.html)

In [ ]:
surface_temp_pattern.weighted(weights).mean().values # should be equal to 1


Plot the normalised warming pattern. Which regions are projected to experience larger relative warming?

3. Scale the pattern by the new global mean temperature change. Add the scaled temperature changes (which assumed ~3K global warming) to the preindustrial temperatures. What's the projected temperature for Oxford in 2100?

## Compare pattern of projected changes to observed changes so far
How does the pattern of surface temperature changes predicted by the UKESM1 climate model compare to observed changes between 1950 and 2025?

You can compare the projected warming pattern to trend in ERA5 reanalysis data, which was already downloaded: ERA5_t2m_monthly_1950-2025_global_lfs.nc. The file is more than 1GB due to ERA5 having much higher spatial resolution than the climate model. Use the first and last 30 year periods to compute a baseline and present-day observed climate. The time scales are different to the climate model projections, but again we can look at normalised patterns to compare the two.

## References
* Santer, B. D., Wigley, T. M., Schlesinger, M. E., & Mitchell, J. F. (1990). Developing climate scenarios from equilibrium GCM results.
* Mitchell, T. D. (2003). Pattern scaling: an examination of the accuracy of the technique for describing future climates. Climatic change, 60(3), 217-242.
* Tebaldi, C., & Arblaster, J. M. (2014). Pattern scaling: Its strengths and limitations, and an update on the latest model simulations. Climatic Change, 122, 459-471.
* IPCC, 2021: Summary for Policymakers. In: Climate Change 2021: The Physical Science Basis. Contribution of Working Group I to the Sixth Assessment Report of the Intergovernmental Panel on Climate Change [Masson-Delmotte, V., P. Zhai, A. Pirani, S.L. Connors, C. Péan, S. Berger, N. Caud, Y. Chen, L. Goldfarb, M.I. Gomis, M. Huang, K. Leitzell, E. Lonnoy, J.B.R. Matthews, T.K. Maycock, T. Waterfield, O. Yelekçi, R. Yu, and B. Zhou (eds.)]. In Press.